In [20]:
#!pip install contractions

import re, string, unicodedata                          # Import Regex, string and unicodedata.
from google.colab import files
import contractions                                     # Import contractions library.
from bs4 import BeautifulSoup                           # Import BeautifulSoup.

import numpy as np                                      # Import numpy.
import pandas as pd                                     # Import pandas.
import nltk                                             # Import Natural Language Tool-Kit.

#nltk.download('stopwords')                              # Download Stopwords.
#nltk.download('punkt')
#nltk.download('wordnet')

from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.
#uploaded = files.upload()

In [21]:
Data = pd.read_csv("Tweets.csv")

In [22]:
Data = Data.loc[:, Data.columns.intersection(['text','airline_sentiment'])]

In [23]:
Data.shape

(14640, 2)

In [24]:
Data.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
